## OpenAI Tools
### in this Demo we will use OpenAI tools to create chatbot for a VOD service to recommend movies to the users based on Genre, Cast and movie Overview.


In [61]:
import os
import openai
import json
from dotenv import load_dotenv, find_dotenv
from pandas.core.computation.expressions import evaluate
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import ast
from openai import AzureOpenAI


In [66]:


_=load_dotenv(find_dotenv()) 

AZURE_ENDPOINT =os.environ["AZURE_OPENAI_ENDPOINT"]
API_KEY= os.environ['OPENAI_API_KEY']
API_VERSION=os.environ['api_version']
OPENAI_DEPLOYMENT=os.environ["OPENAI_DEPLOYMENT"]

client = AzureOpenAI(
  azure_endpoint =AZURE_ENDPOINT, 
  api_key= API_KEY,  
  api_version=API_VERSION
)




## Import and prepare Recommendation data 

In [63]:


credits=pd.read_csv('the-movies-dataset/credits.csv')
movies=pd.read_csv('the-movies-dataset/movies_metadata.csv')

import unicodedata
import re
def clean_data(value):
    value =  unicodedata.normalize('NFD', value).encode('ascii', 'ignore').decode('ascii')
    x= str.lower(re.sub(r"[^a-zA-Z0-9]+", '_', value.strip().replace(" ","_")))
    if(x!='_'):
        return x 
    else:
        return " " 
    
credits['id']=credits['id'].astype('str')
training_df=(credits.merge(movies,on='id'))
training_df['popularity']=training_df['popularity'].astype('float')
training_df=training_df[['title','cast','genres','overview','popularity']]
training_df['overview']=training_df['overview'].str.lower()
training_df['cast']=training_df['cast'].apply(lambda s: list(ast.literal_eval(s)))
training_df['cast'] = training_df['cast'].map(lambda x: x[:3] if len(x) >=4 else x)
training_df['cast']=training_df['cast'].apply((lambda cast : [ clean_data(actor['name'])  for actor in cast  if actor != '' and actor != ' ' ]))
training_df['genres']=training_df['genres'].apply(lambda s: list(ast.literal_eval(s)))
training_df['genres']=training_df['genres'].apply((lambda genres : [ clean_data(genre['name'])  for genre in genres]))


C:\Users\v-akhrisat\AppData\Local\Temp\ipykernel_21068\3260591196.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies=pd.read_csv('the-movies-dataset/movies_metadata.csv')


### Implement function to return the movie recommendation using genres, cast and overview 

In [58]:
def get_top_recommendation(genre,cast=[],overview=None):

    if genre is not None:
        genre_filter = training_df['genres'].map(lambda u: clean_data(genre)  in u)
        if cast is not None:
            cast= [ clean_data(actor)  for actor in cast  if actor != '' and actor != ' ' ]
            cast_filter = training_df['cast'].map(lambda c: set(cast).issubset(c))
        if overview is not None:
            overview_filter =training_df['overview'].str.contains(overview)


        if cast is not None and overview is not None:    
          result=training_df[genre_filter & cast_filter &overview_filter]
        elif  cast is None and overview is not None:  
             result=training_df[genre_filter  &overview_filter]
        elif  cast is not None and overview is  None:  
             result=training_df[genre_filter  &cast_filter   ]
        else : 
             result=training_df[genre_filter]

        if result.shape[0]>=1:
                result= result.sort_values(by=['popularity'],ascending=False).head(10)
                movie_name= result.iloc[np.random.randint(1, 9)]['title'] 
        if movie_name  is not None:
       
                return movie_name
        else:
                return  None

    return None



In [59]:

import random
def get_movie_recommendation(genre,cast=[], overview=''):
    move_name=get_top_recommendation(genre,cast,overview)
    if move_name  is not None:
        return { "movie_name": move_name}
    else:
        return  { "movie_name":'we could not find any movie'}

In [60]:
get_movie_recommendation('Drama')

{'movie_name': 'War for the Planet of the Apes'}

## Define Tool Schema for OpenAI 

In [6]:
# define a function
tools = [
    {
        "type": "function",
        "function": {
        "name": "get_movie_recommendation",
        "description": "use this function only when user ask you for a movie recomendation based on genre, cast or  overview. ",
        "parameters": {
            "type": "object",
            "properties": {
                "genre": {
                    "type": "string",
                    "description": "The movie genre like Sci-Fi, Drama, Romance",
                },
            "cast": {
             "type": "array",
              "items": {
                   "type": "string",
                    "description": "The movie casts member could be more than one actor",
              }}
                , "overview": {
                                    "type": "string",  
                    "description": "part of movie description like War, Race,Family... ",
                            },
                  },
            "required": ["genre"],
      
    }
    }
    }
]


### Sci-FI has typo but OpenAI was able to return the correct spelling

In [7]:

messages = [
    {
        "role": "user",
        "content": "recommend me a Si-Fi movie"
    }
]
response = client.chat.completions.create(
    model=OPENAI_DEPLOYMENT,
    messages=messages,
    tools=tools
)

response.choices

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_cDWcSFVsewimgfoaV6IBOG9r', function=Function(arguments='{"genre":"Sci-Fi"}', name='get_movie_recommendation'), type='function')]), content_filter_results={})]

### Include muliple arguments


In [8]:
# Sci-FI has typo but OpenAI was able to get the correct spelling
messages = [
    {
        "role": "user",
        "content": "recommend me a Sci-Fi movie talks about space"
    }
]
response = client.chat.completions.create(
    model=OPENAI_DEPLOYMENT,
    messages=messages,
    tools=tools
)
response.choices

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_qemCV2lqzIvdScAGVocOHpe5', function=Function(arguments='{"genre":"Sci-Fi","overview":"space"}', name='get_movie_recommendation'), type='function')]), content_filter_results={})]

### Check if the function will be called, if the user didn't request recommendation

In [9]:
# check it the model will call the tool
messages = [
    {
        "role": "user",
        "content": "do you know a famous Sci-Fi movie"
    }
]
response = client.chat.completions.create(
    model=OPENAI_DEPLOYMENT,
    messages=messages,
    tools=tools
)
response.choices

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Yes, one of the most famous Sci-Fi movies is "Blade Runner" directed by Ridley Scott. It\'s a classic from 1982, set in a dystopian future where synthetic humans known as replicants are bio-engineered by powerful corporations. The story follows Rick Deckard, a "blade runner" who is tasked with hunting down and "retiring" rogue replicants. The film is highly regarded for its futuristic vision, philosophical depth, and groundbreaking special effects.', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})]

## Check if the OpenAI will determine the cast full names 

In [458]:
# determine the Actors names
messages = [
    {
        "role": "user",
        "content": "recommend me a Drama movie talks about space has Brad and Angelina acting in it"
    }
]
response = client.chat.completions.create(
    model=OPENAI_DEPLOYMENT,
    messages=messages,
    tools=tools
)
response.choices

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Lmdu4za9UUfeq1gR6Y23cKAX', function=Function(arguments='{"genre":"Drama","cast":["Brad Pitt","Angelina Jolie"],"overview":"space"}', name='get_movie_recommendation'), type='function')]), content_filter_results={})]

## Check if OpenAI will ask for the function  arguments 

In [319]:
# model asking for arguments
messages = [
    {
        "role": "user",
        "content": "recommend me a movie for the night"
    }
]
response = client.chat.completions.create(
    model=OPENAI_DEPLOYMENT,
    messages=messages,
    tools=tools
)
response.choices

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I can help you with that! Could you please provide me with some details like the genre, any favorite actors, or a specific type of storyline or theme you're in the mood for? This will help me give you a more tailored recommendation.", role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})]

## Check if  OpenAI will validate for the function arguments

In [64]:
# Sci-FI has typo but OpenAI was able to get the correct spelling
messages = [
    {
        "role": "user",
        "content": "recommend me a movie with  genre xyz "
    }
]
response = client.chat.completions.create(
    model=OPENAI_DEPLOYMENT,
    messages=messages,
    tools=tools
)
response.choices

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='It looks like "xyz" might not be a valid genre. Could you please provide a specific genre such as Sci-Fi, Drama, Romance, or another genre you\'re interested in? This will help me recommend a movie you\'ll enjoy!', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})]

## Return the function result to the Model

In [10]:
# determine the actor name 
messages = [
    {
        "role": "user",
        "content": "recommend me a Action movie has XXX acting in it"
    }
]
response = client.chat.completions.create(
    model=OPENAI_DEPLOYMENT,
    messages=messages,
    tools=tools
)
response.choices

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_moa6BQq3fouKGZGQFeIbjP5M', function=Function(arguments='{"genre":"Action","cast":["Vin Diesel"]}', name='get_movie_recommendation'), type='function')]), content_filter_results={})]

In [11]:
    response_message = response.choices[0].message
    messages.append(response_message)

    print("Model's response:")  
    print(response_message)  

    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            print(tool_call.function.name)
            if tool_call.function.name == "get_movie_recommendation":
                function_args = json.loads(tool_call.function.arguments)
                print(f"Function arguments: {function_args}")    
                recommendations = get_movie_recommendation(**function_args)['movie_name']
                print(recommendations)
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "atool",
                    "name": "get_movie_recommendation",
                    "content": recommendations,
                })
    else:
        print("No tool calls were made by the model.")  

Model's response:
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_moa6BQq3fouKGZGQFeIbjP5M', function=Function(arguments='{"genre":"Action","cast":["Vin Diesel"]}', name='get_movie_recommendation'), type='function')])
get_movie_recommendation
Function arguments: {'genre': 'Action', 'cast': ['Vin Diesel']}
The Fate of the Furious
